In [3]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn import tree

In [4]:
# Create the connection to the sql server w.r.t to the DNS set
conn = pyodbc.connect("DSN=sq02", autocommit=True)

# SQL query across the tables
sql = """

-- Includes the universal codes cat.[Product Main Group], cat.[Activity Code] AND cat.[Product Sub-Group]
select *
from (
select distinct I.No_ AS itemNo, UPPER(I.codMarchio) AS cM, I.[codReparto] AS cR, I.[codSesso] AS cS, cat.[Sport] AS sp, I.[codTipo] AS cT,
cat.[Vendor No_] AS vNo, UPPER(cat.[Vendor Item No_]) AS vINo, UPPER(cat.[Item Description]) AS iD, cat.[Size Code] AS sC,
		cat.[Activity Code] AS aC, cat.[Product Main Group] AS pMG, cat.[Product Sub-Group] AS pSG, cat.Category
		--, I.[codSport] AS cG, cat.[Color Code] AS cC
   FROM [3A_DWH].[dbo].[DimItem] I
   inner join [3A_STAGE].[nav].[Listini - Details] cat on
       UPPER(I.[Vendor Item No_]) = UPPER(cat.[Vendor Item No_])
       and UPPER(I.codMarchio) = UPPER(cat.Brand) and UPPER(I.[codSesso]) = UPPER(cat.[Sex]) and UPPER(I.[Description]) = UPPER(cat.[Item Description])
       --where cat.Category != ''
	WHERE cat.[Product Main Group] <> '' AND cat.[Activity Code] <> '' AND cat.[Product Sub-Group] <> ''
   group by I.No_, I.[codMarchio], I.[codReparto], I.[codSesso], I.[codSport], I.[codTipo], cat.[Vendor No_],
	cat.[Vendor Item No_], cat.[Item Description], cat.[Sex], cat.[Sport], cat.[Size Code], cat.[Activity Code], cat.[Product Main Group], cat.[Product Sub-Group], cat.Category
   --, I.[codSport], cat.[Color Code]
   )t
   group by itemNo, cM, cR, cS, sp, cT, vNo, vINo, iD, sC, ac, pMG, pSG, category
   --, cG, cC
  
"""

In [5]:
# Read sql query output to panda dataframe
df = pd.io.sql.read_sql(sql, conn)

In [6]:
df.to_csv('dataset/productCategorizer.csv')

In [7]:
df.head()

,itemNo,cM,cR,cS,sp,cT,vNo,vINo,iD,sC,aC,pMG,pSG,Category
0,A03-00000015,A03,4,U,75,SC4,F02124,BB9788,RUNEO 10K,BB9788,75,96,1,
1,A03-00000016,A03,4,J,75,SC4,F02124,BB9950,BB9TIS K,BB9950,75,96,6,
2,A03-00000037,A03,1,U,75,PA1,F02124,BR3699,ESS 3S TCF P FL,BR3699,75,43,1,
3,A03-00000040,A03,1,U,09,TS1,F02124,BS2588,MARKER TS,BS2588,78,33,4,
4,A03-00000049,A03,1,J,00,TU1,F02124,CE8602,YB LINEAR TS CH,CE8602,00,33,3,


In [8]:
# replace all blank/empty cells with NaNs
df.replace('', np.nan, inplace=True)

In [9]:
df.head()

,itemNo,cM,cR,cS,sp,cT,vNo,vINo,iD,sC,aC,pMG,pSG,Category
0,A03-00000015,A03,4,U,75,SC4,F02124,BB9788,RUNEO 10K,BB9788,75,96,1,NaN
1,A03-00000016,A03,4,J,75,SC4,F02124,BB9950,BB9TIS K,BB9950,75,96,6,NaN
2,A03-00000037,A03,1,U,75,PA1,F02124,BR3699,ESS 3S TCF P FL,BR3699,75,43,1,NaN
3,A03-00000040,A03,1,U,09,TS1,F02124,BS2588,MARKER TS,BS2588,78,33,4,NaN
4,A03-00000049,A03,1,J,00,TU1,F02124,CE8602,YB LINEAR TS CH,CE8602,00,33,3,NaN


### Missing values in df
###### https://www.datacamp.com/community/tutorials/categorical-data

In [10]:
print(len(df), '\n', df.isnull().values.sum())

79866 
 19693


In [11]:
# column-wise distribution of null values
print(len(df), '\n\n', df.isnull().sum())

79866 

 itemNo          0
cM              0
cR              0
cS              0
sp              0
cT              0
vNo             0
vINo            0
iD              0
sC              0
aC              0
pMG             0
pSG             0
Category    19693
dtype: int64


### Remove rows with NaN cells from any column except 'Category'

In [12]:
df.dropna(subset = [ x for x in df.columns if x != 'Category'], inplace = True)
print(len(df), '\n\n', df.isnull().sum())

79866 

 itemNo          0
cM              0
cR              0
cS              0
sp              0
cT              0
vNo             0
vINo            0
iD              0
sC              0
aC              0
pMG             0
pSG             0
Category    19693
dtype: int64


### Feature Engineering

In [13]:
### split the iD column into strings by spaces
df_iD = df.copy()
series_iD = df['iD'].str.strip().str.split(' ')

######Derive new array for first and last words on the lists........values changes from series to array to stop the NaN outputs
df_iD['iD_0'] = series_iD.apply(lambda w: w[0]).values
df_iD['iD_L'] = series_iD.apply(lambda w: w[-1]).values

##### View last values of the file => check that NaN values are not registered
df_iD.tail()

,itemNo,cM,cR,cS,sp,cT,vNo,vINo,iD,sC,aC,pMG,pSG,Category,iD_0,iD_L
79861,R04-00000826,R04,4,J,16,SC4,F02124,DV4041,REEBOK ROYAL CLJOG 2 KC,DV4041,46,06,3,RUNNING,REEBOK,KC
79862,R04-00000829,R04,4,J,75,SC4,F02124,DV3871,REEBOK ROYAL PRIME ALT,DV3871,75,96,3,CASUAL,REEBOK,ALT
79863,R04-00000833,R04,4,J,16,SC4,F02124,DV4434,REEBOK RUSH RUNNER,DV4434,46,06,3,RUNNING,REEBOK,RUNNER
79864,R04-00000853,R04,4,J,16,SC4,F02124,DV9029,REEBOK ROYAL CLJOG 2 KC,DV9029,46,06,3,RUNNING,REEBOK,KC
79865,Z99-0032919,A03,4,U,04,SC4,F02124,015110,COPA MUNDIAL,015110,32,15,1,FOOTBALL/SOCCER,COPA,MUNDIAL


In [15]:
df_iD[df_iD['iD_0']==df_iD['iD_L']].head(10)

,itemNo,cM,cR,cS,sp,cT,vNo,vINo,iD,sC,aC,pMG,pSG,Category,iD_0,iD_L
26,A03-00000171,A03,2,X,04,VA2,F02124,AP1091,REPLSTUDSCERAMI,AP1091,32,98,7,FOOTBALL/SOCCER,REPLSTUDSCERAMI,REPLSTUDSCERAMI
98,A03-00000190,A03,2,X,04,PR1,F02124,AP7035,EVERLESTO,AP7035,32,93,6,NaN,EVERLESTO,EVERLESTO
151,A03-00000045,A03,1,U,75,TS1,F02124,CD9242,NUMBER,CD9242,75,12,4,NaN,NUMBER,NUMBER
266,A03-00000171,A03,2,X,04,VA2,F02124,AP1091,REPLSTUDSCERAMI,AP1091,32,98,7,NaN,REPLSTUDSCERAMI,REPLSTUDSCERAMI
282,A03-00000257,A03,2,X,04,PA3,F02124,BS3453,FINALE17JUVECPT,BS3453,32,88,2,NaN,FINALE17JUVECPT,FINALE17JUVECPT
295,A03-00000351,A03,2,X,04,PR1,F02124,AP7036,EVERLESTO,AP7036,32,93,6,NaN,EVERLESTO,EVERLESTO
320,A03-00000308,A03,4,U,75,SC4,F02124,BB9783,10K,BB9783,75,96,1,NaN,10K,10K
351,A03-00000014,A03,4,U,75,SC4,F02124,BB9786,10K,BB9786,75,96,1,NaN,10K,10K
385,A03-00000258,A03,2,X,04,PA3,F02124,BS3456,FINALE17JUVEMIN,BS3456,32,88,2,NaN,FINALE17JUVEMIN,FINALE17JUVEMIN
413,A03-00000415,A03,2,X,04,PA3,F02124,AO4837,EUROPEANQGLI,AO4837,32,88,2,NaN,EUROPEANQGLI,EUROPEANQGLI


### Delete category classes that are very few

In [16]:
## Create a dictionary of Category : value_count
categ_count = dict(zip(df_iD['Category'].value_counts().index, df_iD['Category'].value_counts().values))
### Create list of "Category" whose value_counts is less than 20 or as chosen
elimnate_categs = [x for x in categ_count.keys() if categ_count[x] < 20]
# Deleting rows with Category in elimnate_categs
cleanedData = df_iD[~df_iD.Category.isin(elimnate_categs)]
print ('Perecntage remaining after deletion: ', len(cleanedData)/len(df_iD))

Perecntage remaining after deletion:  0.9995242030400922


## Transform Categorical features to numerical

In [17]:
#####   Transform Categorical features to numerical
# w is the dataframe with categorical values
def categ_maps(w):
    mapping_dict = {}
    for x, y in enumerate(w.columns):
        d = {}
        for a,b in enumerate(w.iloc[:,x].unique()):
            d.update({b: a})
        mapping_dict.update({y: d})
    return mapping_dict
# The function to map dataframe to numerical values
def df_maps(z):
    g = categ_maps(z)
    df_mapped = pd.DataFrame()
    for r in z.columns:
        df_mapped[r] = z[r].map(g[r])
    return df_mapped

In [18]:
# Using df_maps created function, transform the data
df_num = df_maps(cleanedData)
print (len(cleanedData), len(df_num), '\n\n', df_num.head(3))

79828 79828 

    itemNo  cM  cR  cS  sp  cT  vNo  vINo  iD  sC  aC  pMG  pSG  Category  \
0       0   0   0   0   0   0    0     0   0   0   0    0    0         0   
1       1   0   0   1   0   0    0     1   1   1   0    0    1         0   
2       2   0   1   0   0   1    0     2   2   2   0    1    0         0   

   iD_0  iD_L  
0     0     0  
1     1     1  
2     2     2  


In [19]:
### Test functions
dic_obj = categ_maps(cleanedData)
print (dic_obj.keys(), '\n')
print (dic_obj['Category'], '\n\n')
### Number of data with Null values and those with values
print ('Null w.r.t numerical data : ', len(df_num[df_num['Category']==dic_obj['Category'][np.nan]]),\
       '\n', 'Null w.r.t original data : ', len(cleanedData[cleanedData['Category'].isnull()]))

dict_keys(['itemNo', 'cM', 'cR', 'cS', 'sp', 'cT', 'vNo', 'vINo', 'iD', 'sC', 'aC', 'pMG', 'pSG', 'Category', 'iD_0', 'iD_L']) 

{nan: 0, 'NOT SPORTS SPECIFIC': 1, 'FOOTBALL/SOCCER': 2, 'BASKETBALL': 3, 'TENNIS': 4, 'SWIM': 5, 'TRAINING': 6, 'RUNNING': 7, 'RUGBY': 8, 'OUTDOOR': 9, 'SKATEBOARDING': 10, 'ORIGINALS': 11, 'NIKE SPORTSWEAR': 12, 'YOUNG ATHLETES': 13, 'FOOTBALL, BASEBALL, AT': 14, 'WOMEN TRAINING': 15, 'ACTION OUTDOOR': 16, 'OTHER': 17, 'CASUAL': 18} 


Null w.r.t numerical data :  19693 
 Null w.r.t original data :  19693


In [20]:
scaledLabelledData = df_num[df_num['Category'] != 0]
scaledUnlabelledData = df_num[df_num['Category'] == 0]

In [21]:
print (len(scaledLabelledData), len(scaledUnlabelledData))

60135 19693


## Split dataset

In [22]:
x_train, x_test, y_train, y_test = train_test_split(scaledLabelledData.drop(['Category'], axis=1), scaledLabelledData.loc\
                                                    [:,'Category'], test_size=0.25, random_state=0)

## Decision tree

In [23]:
prod = tree.DecisionTreeClassifier()
prod = prod.fit(x_train, y_train)

In [24]:
# Use fit_predict to fit model and obtain cluster labels: labels
pred_labels = prod.predict(x_test)

# Create a DataFrame with clusters and varieties as columns: df
dfpy = pd.DataFrame({'pred_labels': pred_labels, 'y_labels': y_test})

# Create crosstab: ct
ct = pd.crosstab(dfpy['pred_labels'], dfpy['y_labels'])

# Display ct
print(ct)

y_labels      1     2    3   4   5    6     7   8   9   10  11    12    13  \
pred_labels                                                                  
1            399     0    1   0   0    5     0   0   0   0   0     0     0   
2              0  2604    0   0   0    0     0   0   0   0   0     5     0   
3              0     0  558   0   0    0     0   0   0   0   0     0     1   
4              0     0    0  67   0    0     1   0   0   0   1     0     0   
5              0     0    0   0  70    0     0   0   0   0   0     0     0   
6              3     0    0   0   0  233     0   0   0   0   0     0     0   
7              0     0    0   0   0    0  1958   0   0   0   0     3     0   
8              0     0    0   0   0    0     0  10   0   0   0     0     0   
9              0     0    0   0   0    0     0   0   3   0   0     0     0   
10             0     0    0   0   0    0     0   0   0  10   0     0     0   
11             0     0    0   0   1    0     0   0   0   0  29  

## Retrain and perform unlabelled predictions

In [25]:
prod_obj = tree.DecisionTreeClassifier()
prod_model = prod_obj.fit(scaledLabelledData.drop(['Category'], axis=1), scaledLabelledData.loc[:,'Category'])

In [26]:
predicted_unlabeled = prod_model.predict(scaledUnlabelledData.drop(['Category'], axis=1))

In [27]:
# Invert the dictionary
dic_keys = dic_obj['Category'].keys()
dic_values = dic_obj['Category'].values()
dic_inv = dict(zip(dic_values, dic_keys))

In [28]:
predictions = [dic_inv[key] for key in predicted_unlabeled]
#print (predicted_unlabeled, predictions)
print (len(predictions), len(predicted_unlabeled), len(cleanedData[cleanedData['Category'].isnull()]))

19693 19693 19693


In [29]:
df_predictions = cleanedData[cleanedData['Category'].isnull()].drop('Category', axis = 1)
df_predictions['Categ_ID'] = predicted_unlabeled
df_predictions['Category'] = predictions
df_predictions.tail(3)

,itemNo,cM,cR,cS,sp,cT,vNo,vINo,iD,sC,aC,pMG,pSG,iD_0,iD_L,Categ_ID,Category
79805,R04-00000423,R04,4,U,75,SC4,F02124,BS9088,REEBOK ROYAL TECHQUE T,BS9088,75,96,1,REEBOK,T,4,TENNIS
79811,R04-00000466,R04,4,J,16,SC4,F02124,BS8014,REEBOK ROYAL CLJOG 2,BS8014,46,06,3,REEBOK,2,7,RUNNING
79820,R04-00000532,R04,2,X,00,VA2,F02124,BK3386,FOUND BOTTLE 500,BK3386,00,86,2,FOUND,500,6,TRAINING


### Adjust output according to agent reviews

#### LORIS

In [38]:
# LORIS (cM=J11) advises that every column be labelled "YOUNG ATHLETES" ... Thereby replacing "OTHER" and "NIKE SPORTSWEAR"
df_Loris = df_predictions.copy()

In [31]:
cM_boolLo = (df_Loris['cM']=='J11') | (df_Loris['cM']=='N29') # Loris boolean list w.r.t J11 and N29
categ_boolLo = (df_Loris['Category']!='YOUNG ATHLETES') # Loris boolean list w.r.t NOT 'YOUNG ATHLETES'
no_categsLo = [x for x in dic_keys if x != 'YOUNG ATHLETES'] # List of all other categories but not 'YOUNG ATHLETES'
Loris = df_Loris[cM_boolLo & categ_boolLo]['Category'].replace(no_categsLo, "YOUNG ATHLETES", inplace=False)
df_Loris.update(Loris)

In [32]:
print (df_Loris[cM_boolLo & categ_boolLo].loc[:, ['cM', 'Categ_ID', 'Category']].drop_duplicates())

        cM  Categ_ID        Category
18117  N29        12  YOUNG ATHLETES
18571  J11         7  YOUNG ATHLETES
19899  J11        12  YOUNG ATHLETES


## Confirm an optional solution with Loris
* In this approach, for N29 I replace only a selected list of categories BUT replacing every categ in list J11 that aren't 'YOUNG ATHLETES'

In [59]:
df_Loris1 = df_predictions.copy()
boolLo_J11 = (df_Loris1['cM']=='J11') & (df_Loris1['Category'] != 'YOUNG ATHLETES')
boolLo_N29 = (df_Loris1['cM']=='N29') & (df_Loris1['Category'] == 'NIKE SPORTSWEAR')

In [49]:
notCategs_J11 = [x for x in dic_keys if x != 'YOUNG ATHLETES']
notCategs_N29 = [x for x in dic_keys if x == 'NIKE SPORTSWEAR' or x == 'TENNIS' or x == 'OTHER']

In [50]:
serCateg_J11 = df_Loris1[boolLo_J11]['Category'].replace(notCategs_J11, "YOUNG ATHLETES", inplace=False)
serCateg_N29 = df_Loris1[boolLo_N29]['Category'].replace(notCategs_N29, "YOUNG ATHLETES", inplace=False)

In [51]:
df_Loris1.update(serCateg_J11)
df_Loris1.update(serCateg_N29)

In [57]:
print (df_Loris1[boolLo_J11].loc[:, ['cM', 'Categ_ID', 'Category']].drop_duplicates())
print ()
print (df_Loris1[boolLo_N29].loc[:, ['cM', 'Categ_ID', 'Category']].drop_duplicates())
print ()
print (df_Loris1[(df_Loris1['cM']=='J11')|(df_Loris1['cM']=='N29')].loc[:, ['cM', 'Categ_ID', 'Category']].drop_duplicates())

        cM  Categ_ID        Category
18571  J11         7  YOUNG ATHLETES
19899  J11        12  YOUNG ATHLETES

        cM  Categ_ID        Category
18117  N29        12  YOUNG ATHLETES

        cM  Categ_ID        Category
18111  J11        13  YOUNG ATHLETES
18117  N29        12  YOUNG ATHLETES
18118  N29        13  YOUNG ATHLETES
18571  J11         7  YOUNG ATHLETES
19899  J11        12  YOUNG ATHLETES


### Ezio

In [33]:
# Ezio (cM=A03, R04) advises that every column "TENNIS" and "NIKE SPORTSWEAR" be replaced with "CORE"
df_Ezio = df_Loris.copy()

In [34]:
cM_boolEz = (df_Ezio['cM']=='A03') | (df_Ezio['cM']=='R04')
categ_boolEz = (df_Ezio['Category']=='TENNIS')|(df_Ezio['Category']=='NIKE SPORTSWEAR')|(df_Ezio['Category']=='OTHER')
Ezio = df_Ezio[cM_boolEz & categ_boolEz]['Category'].replace(["TENNIS", "NIKE SPORTSWEAR"], "CORE", inplace=False)
df_Ezio.update(Ezio)

In [36]:
print (df_Ezio[cM_boolEz & categ_boolEz].loc[:, ['cM', 'Categ_ID', 'Category']].drop_duplicates())

        cM  Categ_ID Category
1      A03         4     CORE
8100   R04         4     CORE
16172  A03        12     CORE


### Obtain dataframe for Category dictionary

In [58]:
# dataframe of category dictionary
df_dic = pd.DataFrame({'categ_ID':list(dic_values), 'category':list(dic_keys)}).iloc[1:, ]
df_dict = df_dic.append({'categ_ID': [dic_obj['Category']['TENNIS'], dic_obj['Category']['NIKE SPORTSWEAR']],\
                         'category': 'CORE'}, ignore_index=True)
print (df_dict)

   categ_ID                category
0         1     NOT SPORTS SPECIFIC
1         2         FOOTBALL/SOCCER
2         3              BASKETBALL
3         4                  TENNIS
4         5                    SWIM
5         6                TRAINING
6         7                 RUNNING
7         8                   RUGBY
8         9                 OUTDOOR
9        10           SKATEBOARDING
10       11               ORIGINALS
11       12         NIKE SPORTSWEAR
12       13          YOUNG ATHLETES
13       14  FOOTBALL, BASEBALL, AT
14       15          WOMEN TRAINING
15       16          ACTION OUTDOOR
16       17                   OTHER
17       18                  CASUAL
18  [4, 12]                    CORE


### Save outputs

In [60]:
print ("Outputing result(s)")

Outputing result(s)


In [61]:
writer = pd.ExcelWriter('output/productCategorized.xlsx')
df_Ezio.to_excel(writer,'productCategory')
ct.to_excel(writer,'accuracyTest')
df_dict.to_excel(writer,'dictionaryCategory')
writer.save()

In [62]:
print ("Experiment Successfully Accomplished")

Experiment Successfully Accomplished


In [64]:
!dir

 Il volume nell'unit… C non ha etichetta.
 Numero di serie del volume: 3660-697C

 Directory di C:\Users\festo.owiny\Desktop\Models\productCategorization

19/11/2018  10:16    <DIR>          .
19/11/2018  10:16    <DIR>          ..
19/11/2018  08:56    <DIR>          .ipynb_checkpoints
19/11/2018  08:57    <DIR>          dataset
19/11/2018  10:15    <DIR>          output
19/11/2018  10:16            54.013 productCategorizer.ipynb
16/11/2018  16:39             8.658 productCategorizer.py
16/10/2018  14:40               164 run.bat
               3 File         62.835 byte
               5 Directory  384.395.206.656 byte disponibili
